# Instalação e importação das bibliotecas

In [2]:
!pip install numpy pandas matplotlib scikit-learn opencv-python tensorflow keras opencv-python-headless

In [3]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-09-29 17:18:36.391059: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-29 17:18:36.538217: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-29 17:18:36.685097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-29 17:18:36.812679: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-29 17:18:36.849504: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-29 17:18:37.088288: I tensorflow/core/platform/cpu_feature_gu

#### Apenas um aviso informando que não foram encontrados drivers cuda e por conta disso será usada a CPU da máquina ao invés da GPU

# Coleta e processamento de dados

In [4]:
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, (224, 224))
    img_normalized = img_resized / 255.0
    return img_normalized

### Criando CNN (rede neural convolucional)
#### Essa função cria um modelo CNN que passa por várias camadas convolucionais para extrair características das imagens, nas quais foram utilizados a função ReLU (função de ativação que permite que a rede neural aprenda padrões complexos) e o Pooling (operação de amostragem usada em redes convolucionais para reduzir as dimensões e/ou tamanho da imagem ou das saídas das camadas convolucionais, porém, preservando suas características mais importantes) seguida por uma camada densa para classificação final em 3 classes, com a função de ativação softmax com 3 neurônios que transforma a saída em uma distribuição de probabilidades (somando 1), apropriada para tarefas de classificação multiclasses (neste caso, classificando em 3 classes possíveis).

In [5]:
def create_cnn(input_shape):
    model = models.Sequential()

    model.add(layers.Input(shape=input_shape)) # Definie a forma de entrada

    # Camada convolucional + ReLU + Pooling
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())  # Achata a saída para alimentar a camada densa
    model.add(layers.Dense(128, activation='relu'))  # Camada densa
    model.add(layers.Dense(3, activation='softmax'))

    return model

# Definindo a forma de entrada da imagem (altura, largura, canais)
input_shape = (224, 224, 1)  # 1 canal para imagens em escala de cinza
cnn_model = create_cnn(input_shape)

# Compilando o modelo
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Treinamento do modelo

In [8]:
# Gerador de dados com aumentação de imagem
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.5
)

# Diretório das imagens de treino e validação
train_generator = train_datagen.flow_from_directory(
    './data_set',
    target_size=(224, 224),
    color_mode='grayscale',
    batch_size=1, 
    # aumenta o tamanho de processamento por vez, reduzindo peculiaridades de uma única imagem afetem a acurácia do modelo
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    './data_set',
    target_size=(224, 224),
    color_mode='grayscale',
    batch_size=1, # mudar para 8 quando tiver dataset completo
    # aumenta o tamanho de processamento por vez, reduzindo peculiaridades de uma única imagem afetem a acurácia do modelo
    class_mode='categorical',
    subset='validation'
)

# Treinamento do modelo
history = cnn_model.fit(train_generator, validation_data=validation_generator, epochs=30)

Found 3 images belonging to 3 classes.
Found 3 images belonging to 3 classes.
Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 541ms/step - accuracy: 0.2917 - loss: 0.9769 - val_accuracy: 0.3333 - val_loss: 1.8361
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 493ms/step - accuracy: 0.1667 - loss: 0.9869 - val_accuracy: 0.0000e+00 - val_loss: 2.0048
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 470ms/step - accuracy: 0.5417 - loss: 0.8793 - val_accuracy: 0.0000e+00 - val_loss: 1.4120
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 668ms/step - accuracy: 0.7083 - loss: 0.9622 - val_accuracy: 0.3333 - val_loss: 0.9172
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 434ms/step - accuracy: 1.0000 - loss: 0.5341 - val_accuracy: 0.3333 - val_loss: 1.6441
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 424ms/step - accuracy: 0.2917 - loss: 0.8025 - val_accuracy: 0.6667 - val_loss: 0.8611
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 397ms/step - accuracy: 0.2917 - loss: 0.9614 - val_accuracy: 0.3333 - val_loss: 1.2127
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━

### Ao inicializar o modelo os pesos da rede neural são inicializados de forma aleatória, e como o processo de otimização do modelo começa a partir de diferentes pontos cada execução pode levar a resultados diferentes, por isso, cada vez que o código for executado os resultados não serão exatamente os mesmos, mas aproximados.


### image augmentation: técnicas de rotação, deslocamento, zoom e etc. aplicadas na imagem para criar novas variações para realizar o treinamento do modelo, sendo também um process

Testes de validação

In [9]:
# Avaliando o modelo no conjunto de validação
val_loss, val_acc = cnn_model.evaluate(validation_generator)
print(f"Validação - Loss: {val_loss}, Acurácia: {val_acc}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.1667 - loss: 1.5009  
Validação - Loss: 1.5385065078735352, Acurácia: 0.3333333432674408


Função de predição

In [12]:
def predict_image(image_path, model):
    img = preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)  # Adiciona uma dimensão para batch
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)  # Retorna o índice da classe com maior probabilidade
    return predicted_class, prediction

predicted_class, result = predict_image('./test-data/001.jpg', cnn_model)
print(f'Predição: {result}')
print(f'Classe prevista: {predicted_class}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Predição: [[0.22090256 0.29052022 0.48857716]]
Classe prevista: 2
